In [14]:
# -------------------------------
# Importation des librairies
# -------------------------------
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
# import de Input et Model
from tensorflow.keras import Input, Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense



In [15]:
# Configuration pour utiliser un seul GPU ou CPU
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    print(f"🖥️ GPU détecté : {physical_devices[0]}")
else:
    print("💻 Utilisation du CPU")

# Style pour les visualisations
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

🖥️ GPU détecté : PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [16]:
# -------------------------------
# Chargement et préparation des données
# -------------------------------
data = load_iris()
X = data.data          # 4 caractéristiques (longueur/largeur pétales, sépales)
y = data.target         # 3 classes de fleurs

# Normalisation des données
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Découpage en train / test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [17]:
# -------------------------------
# Création du modèle ANN
# -------------------------------
model = Sequential([
    Dense(8, activation='relu', input_shape=(4,)),   # Couche d'input +  couche cachée 1
    Dense(8, activation='relu'),                     # Couche cachée 2
    Dense(3, activation='softmax')                   # Couche de sortie (3 classes)
])

# Creation du modèle ANN en Seqeuential
def create_model():
    model = Sequential()
    model.add(Dense(8, activation='relu', input_shape=(4,)))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    return model

# Creation du modèle ANN
def create_model():
    x = Input(shape=(4,))
    x1 = Dense(8, activation='relu')(x)
    x2 = Dense(8, activation='relu')(x1)
    output = Dense(3, activation='softmax')(x2)
    return Model(inputs=x, outputs=output)

/opt/homebrew/Caskroom/miniconda/base/envs/alyra-ml-py3.11/lib/python3.11/site-packages/keras/src/layers/core/dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Explication des Couches :

# Input (4 entrées)	: 
## Rôle : Reçoit les 4 caractéristiques du dataset Iris	
## Pourquoi ce choix : Nombre fixé par le nombre de features du dataset (longueur, largeur pétales/sépales)

# 1re couche cachée (8 neurones)
## Rôle : Combine les entrées pour extraire des relations simples
## Pourquoi ce choix : 8 neurones = petit modèle rapide à entraîner, suffisant pour données simples

# 2e couche cachée (8 neurones)	
## Rôle : Apprend des relations plus abstraites (non linéaires)
## Pourquoi ce choix : Ajouter de la profondeur permet d’apprendre des interactions plus complexes

# Couche de sortie (3 neurones)
## Rôle : Donne les probabilités pour chaque classe (Setosa, Versicolor, Virginica)
## Pourquoi ce choix : Une sortie par classe + fonction softmax → proba totale = 1

# Idéé clé :
## Une couche cachée apprend des motifs simples
## Plusieurs couches enchaînées apprennent des représentations de plus en plus abstraites

# Analogie :
## C’est comme une équipe : 
## la 1ʳᵉ couche “observe les détails”,
## la 2ᵉ “fait des associations”
## la dernière “prend la décision finale”.

In [20]:
# Les fonctions d’activation transforment la sortie de chaque neurone pour :
## introduire de la non-linéarité,
## permettre au modèle d’apprendre des relations complexes (non proportionnelles).

# Avantages de ReLU	→ Pourquoi elle est souvent choisie
## Simple et rapide à calculer	→ permet un apprentissage plus rapide
## Ne sature pas pour les valeurs positives	→ évite le problème de “vanishing gradient”
## Donne des sorties nulles pour z < 0	→ favorise la parcimonie (neurones inactifs inutilement)

# Analogie :
## C’est un filtre intelligent : si le signal est fort → il passe, sinon → on l’ignore.

In [21]:
# Rôle                                                                | Pourquoi c’est le bon choix                                |
# | ----------------------------------------------------------------- | ---------------------------------------------------------- |
# | Convertit les sorties en probabilités normalisées (somme = 1)     | → Interprétable comme “proba d’appartenir à chaque classe” |
# | Permet d’utiliser la cross-entropy comme fonction de coût adaptée | → Standard pour classification multi-classes               |

# Analogie
## C’est une course entre les classes : 
## chaque sortie représente la “force” d’une classe
## et Softmax les transforme en pourcentages qui totalisent 100 %.

In [22]:
# -------------------------------
# Compilation du modèle
# -------------------------------
model.compile(
    optimizer='adam',             # Optimiseur : Adam = SGD amélioré
    loss='sparse_categorical_crossentropy',  # Fonction de coût adaptée aux classes entières
    metrics=['accuracy']          # On suit la précision
)

In [23]:
# -------------------------------
# Entraînement
# -------------------------------
history = model.fit(X_train, y_train, epochs=50, batch_size=8, validation_split=0.2, verbose=0)

2025-10-28 20:08:43.148052: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


In [24]:
# -------------------------------
# Évaluation
# -------------------------------
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Précision sur le test : {acc:.2f}")

Précision sur le test : 0.97


In [ ]:
# -------------------------------
# Prédiction
# -------------------------------
sample = np.array([[5.1, 3.5, 1.4, 0.2]])
sample_scaled = scaler.transform(sample)

pred = model.predict(sample_scaled)
print("Probabilités :", np.round(pred,4))
print("Classe prédite :", np.argmax(pred))

if np.argmax(pred) == 0:
    print("Fleur prédite : Setosa")
elif np.argmax(pred) == 1:
    print("Fleur prédite : Versicolor")
else:
    print("Fleur prédite : Virginica")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Probabilités : [[0.9969 0.0031 0.    ]]
Classe prédite : 0
Fleur prédite : Setosa
